# Grid Search

In [1]:
import numpy as np
import tensorflow as tf
import itertools
import pandas as pd
# Own library
import test3
# Needed to clean memory
import gc

# To modify scipts remotly
%load_ext autoreload
%autoreload 2

In [2]:
np.random.seed(94)
tf.random.set_seed(94)
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_test = x_test / 255.0
x_test, y_test = tf.constant(x_test), tf.constant(y_test)

In [3]:
n = 10000
x_test = x_test[:n]
y_test = tf.squeeze(tf.one_hot(y_test[:n], 10))

In [4]:
path = 'data/model_cifar.h5'
model = tf.keras.models.load_model(path)

In [4]:
np.array(range(1, 10)) * 1/255

array([0.00392157, 0.00784314, 0.01176471, 0.01568627, 0.01960784,
       0.02352941, 0.02745098, 0.03137255, 0.03529412])

In [10]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 3s 301us/sample - loss: 1.0237 - accuracy: 0.7091


[1.023684347820282, 0.7091]

In [26]:
# Parameter space
list_eta = np.array(range(1, 10)) * 1/255
list_epsilon = np.array(range(1, 10)) * 1/255
list_n_steps = np.append(np.arange(2, 10, 2), 1)
list_losses = [tf.keras.losses.MeanSquaredError(),
               tf.keras.losses.CategoricalCrossentropy()]

list_methods = [test3.pgd_infinity, test3.pgd_ininity_random, test3.pgd_l2, test3.pgd_l2_random]

# Generate parameters grid
iter_parameters_grid = itertools.product(list_methods, list_eta, list_epsilon, list_n_steps, list_losses)


# Names of methods
list_method_names = ['pgd_infinity', 'pgd_infinity_random', 'pgd_l2', 'pgd_l2_random']

In [27]:
# print(len(list(iter_parameters_grid)))

3240

In [5]:
# Compute different attack sets

nested_accuracies = []
for method, eta, eps, n_steps, loss in iter_parameters_grid:
    if eta >= eps:
        # Does not make sense to test this parameters config
        nested_accuracies.append([np.nan, np.nan])
        continue
    
    x_adv = method(x_test, y_test, model, loss_fn=loss, eta=eta, eps=eps, n_steps=n_steps)
    # Re-scaling image (they are normalized)
    x_adv = tf.clip_by_value(x_adv, 0, 1)
    list_eval = model.evaluate(x=x_adv, y=y_test)
    # Clear memory
    gc.collect()
    tf.keras.backend.clear_session()
    del x_adv
      
    nested_accuracies.append(list_eval)

2/2 [==============================] - 0s 11ms/sample - loss: 0.2290 - accuracy: 1.0000


2/2 [==============================] - 0s 18ms/sample - loss: 2.4126 - accuracy: 0.0000e+00


2/2 [==============================] - 0s 11ms/sample - loss: 6.6249 - accuracy: 0.0000e+00


2/2 [==============================] - 0s 13ms/sample - loss: 0.2022 - accuracy: 1.0000


2/2 [==============================] - 0s 15ms/sample - loss: 2.2839 - accuracy: 0.0000e+00
tf.Tensor([5.0169712  4.98998998], shape=(2,), dtype=float64)
tf.Tensor([0.19932345 0.2004012 ], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 10ms/sample - loss: 6.8785e-04 - accuracy: 1.0000
tf.Tensor([5.0169712  4.98998998], shape=(2,), dtype=float64)
tf.Tensor([0.19932345 0.2004012 ], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 13ms/sample - loss: 5.7659e-04 - accuracy: 1.0000
tf.Tensor([5.0169712  4.98998998], shape=(2,), dtype=float64)
tf.Tensor([0.19932345 0.2004012 ], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 11ms/sample - loss: 5.7659e-04 - accuracy: 1.0000
tf.Tensor([5.0169712  4.98998998], shape=(2,), dtype=float64)
tf.Tensor([0.19932345 0.2004012 ], shape=(2,), dtype=float64)
tf.Tensor([5.0284398  5.16750106], shape=(2,), dtype=float64)
tf.Tensor([0.19886884 0.19351713], shape=(2,), dtype=float64)
tf.Tensor([4

...

tf.Tensor([0.00501697 0.004993  ], shape=(2,), dtype=float64)
tf.Tensor([0.78166058 0.78541408], shape=(2,), dtype=float64)
tf.Tensor([0.00115107 0.00116956], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.00393064 0.00397583], shape=(2,), dtype=float64)
tf.Tensor([0.99769097 0.98635295], shape=(2,), dtype=float64)
tf.Tensor([0.00120642 0.00146151], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.00394078 0.00403461], shape=(2,), dtype=float64)
tf.Tensor([0.99512617 0.97198211], shape=(2,), dtype=float64)
tf.Tensor([0.00129838 0.0018544 ], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.00395079 0.00408976], shape=(2,), dtype=float64)
tf.Tensor([0.99260247 0.95887451], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 10ms/sample - loss: 0.1304 - accuracy: 1.0000
tf.Tensor([0.00501697 0.00497795], shape=(2,), dtype=float64)
tf.Tensor([0.78166058 0.787787

tf.Tensor([0.19697833 0.19697833], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.1552862  0.14922586], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.27760164 0.27177914], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.28721307 0.26539489], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.38252887 0.36944007], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 23ms/sample - loss: 4.6869 - accuracy: 0.0000e+00
tf.Tensor([0.19697833 0.19686118], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.37318831 0.15050856], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.44080594 0.27936876], shape=(2,), dtype=float64)
tf.Tensor([1. 1.], shape=(2,), dtype=float64)
tf.Tensor([0.55431619 0.28732014], shape=(2,

tf.Tensor([0.19697833 0.19639191], shape=(2,), dtype=float64)
tf.Tensor([0.50767006 0.50918595], shape=(2,), dtype=float64)
tf.Tensor([0.14380037 0.13693351], shape=(2,), dtype=float64)
tf.Tensor([0.69540852 0.73028143], shape=(2,), dtype=float64)
tf.Tensor([0.18216528 0.18068136], shape=(2,), dtype=float64)
tf.Tensor([0.54895203 0.55346053], shape=(2,), dtype=float64)
tf.Tensor([0.18960798 0.18438989], shape=(2,), dtype=float64)
tf.Tensor([0.52740396 0.54232909], shape=(2,), dtype=float64)
tf.Tensor([0.19098649 0.18576323], shape=(2,), dtype=float64)
tf.Tensor([0.52359725 0.53831967], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 5ms/sample - loss: 0.0935 - accuracy: 1.0000
tf.Tensor([0.19697833 0.19662668], shape=(2,), dtype=float64)
tf.Tensor([0.50767006 0.50857797], shape=(2,), dtype=float64)
tf.Tensor([0.1476577  0.13556586], shape=(2,), dtype=float64)
tf.Tensor([0.67724201 0.73764885], shape=(2,), dtype=float64)
tf.Tensor([0.17439738 0.16419142], shape=(2,)

tf.Tensor([0.19697833 0.19544997], shape=(2,), dtype=float64)
tf.Tensor([0.05076701 0.05116399], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 19ms/sample - loss: 0.1294 - accuracy: 1.0000
tf.Tensor([0.19674397 0.19592151], shape=(2,), dtype=float64)
tf.Tensor([0.05082748 0.05104085], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 14ms/sample - loss: 0.1157 - accuracy: 1.0000
tf.Tensor([0.19674397 0.19592151], shape=(2,), dtype=float64)
tf.Tensor([0.05082748 0.05104085], shape=(2,), dtype=float64)
2/2 [==============================] - 0s 2ms/sample - loss: 0.1244 - accuracy: 1.0000
tf.Tensor([0.19697833 0.19592151], shape=(2,), dtype=float64)
tf.Tensor([0.05076701 0.05104085], shape=(2,), dtype=float64)
tf.Tensor([0.18787807 0.18694881], shape=(2,), dtype=float64)
tf.Tensor([0.05322601 0.05349058], shape=(2,), dtype=float64)
tf.Tensor([0.18775967 0.18650599], shape=(2,), dtype=float64)
tf.Tensor([0.05325957 0.05361758], shape=(2,), dtype=fl

tf.Tensor([0.19697833 0.19592151], shape=(2,), dtype=float64)
tf.Tensor([0.05076701 0.05104085], shape=(2,), dtype=float64)
tf.Tensor([0.18757031 0.18640218], shape=(2,), dtype=float64)
tf.Tensor([0.05331334 0.05364744], shape=(2,), dtype=float64)
tf.Tensor([0.18718734 0.18636916], shape=(2,), dtype=float64)
tf.Tensor([0.05342242 0.05365695], shape=(2,), dtype=float64)
tf.Tensor([0.187192   0.18637227], shape=(2,), dtype=float64)
tf.Tensor([0.05342109 0.05365605], shape=(2,), dtype=float64)
tf.Tensor([0.18719231 0.18637245], shape=(2,), dtype=float64)
tf.Tensor([0.053421 0.053656], shape=(2,), dtype=float64)
tf.Tensor([0.18719233 0.18637246], shape=(2,), dtype=float64)
tf.Tensor([0.05342099 0.053656  ], shape=(2,), dtype=float64)
tf.Tensor([0.18719233 0.18637246], shape=(2,), dtype=float64)
tf.Tensor([0.05342099 0.053656  ], shape=(2,), dtype=float64)
tf.Tensor([0.18719233 0.18637246], shape=(2,), dtype=float64)
tf.Tensor([0.05342099 0.053656  ], shape=(2,), dtype=float64)
tf.Tensor([0

In [6]:
index = pd.MultiIndex.from_product([list_method_names, list_eta, list_epsilon],
                                   names=['method', 'tau', 'epsilon'])

In [7]:
# Graphs when step size is big
index = pd.MultiIndex.from_product([list_method_names, list_eta, list_epsilon,
                                    list_n_steps, [loss.name for loss in list_losses]],
                                   names=['method', 'eta', 'epsilon', 'n_steps', 'loss_name'])

df_results = pd.DataFrame(data=nested_accuracies,
                          columns=['loss', 'accuracy'],
                          index=index)

In [8]:
df_results.dropna()

loss  \
method        eta      epsilon n_steps loss_name                                
pgd_infinity  0.100000 1.00    1       mean_squared_error           11.348354   
                                       categorical_crossentropy     11.317334   
                                       kullback_leibler_divergence  11.317334   
                               3       mean_squared_error           36.156952   
                                       categorical_crossentropy     38.469917   
...                                                                       ...   
pgd_l2_random 0.003922 0.01    7       categorical_crossentropy      0.120049   
                                       kullback_leibler_divergence   0.130449   
                               9       mean_squared_error            0.118663   
                                       categorical_crossentropy      0.131604   
                                       kullback_leibler_divergence   0.122764   

                                                                    accuracy  
method        eta      epsilon n_steps loss_name                              
pgd_infinity  0.100000 1.00    1       mean_squared_error                0.0  
                                       categorical_crossentropy          0.0  
                                       kullback_leibler_divergence       0.0  
                               3       mean_squared_error                0.0  
                                       categorical_crossentropy          0.0  
...                                                                      ...  
pgd_l2_random 0.003922 0.01    7       categorical_crossentropy          1.0  
                                       kullback_leibler_divergence       1.0  
                               9       mean_squared_error                1.0  
                                       categorical_crossentropy          1.0  
                                       kullback_leibler_divergence       1.0  

[900 rows x 2 columns]

In [9]:
path = './data/df_results.pickle'
import pickle
with open(path, 'wb') as f:
    pickle.dump(df_results, f)